<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/krosnicki/ML_Salary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Salary_Data.csv to Salary_Data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/content/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import HuberRegressor, LinearRegression

In [ ]:
czy_puste_wartosci_w_kolumnach = df.isna().count()
print(czy_puste_wartosci_w_kolumnach)


Age                    375
Gender                 375
Education Level        375
Job Title              375
Years of Experience    375
Salary                 375
dtype: int64


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,1,159,5.0,90000.0
1,28.0,0,2,17,3.0,65000.0
2,45.0,1,3,130,15.0,150000.0
3,36.0,0,1,101,7.0,60000.0
4,52.0,1,2,22,20.0,200000.0


In [ ]:
df.shape

(375, 6)

In [ ]:
df.columns

Index(['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience',
       'Salary'],
      dtype='object')

In [ ]:
import numpy as np

np.sum(df["Salary"].isna())

2

In [ ]:
df.dropna(inplace=True)

In [ ]:
np.sum(df["Salary"].isna())

0

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,1,159,5.0,90000.0
1,28.0,0,2,17,3.0,65000.0
2,45.0,1,3,130,15.0,150000.0
3,36.0,0,1,101,7.0,60000.0
4,52.0,1,2,22,20.0,200000.0


In [ ]:
df['Gender'] = (df['Gender']).astype('string')
df['Education Level'] = (df['Education Level']).astype('string')
df['Job Title'] = (df['Job Title']).astype('string')

In [ ]:
df.dtypes

Age                    float64
Gender                  string
Education Level         string
Job Title               string
Years of Experience    float64
Salary                 float64
dtype: object

In [ ]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,1,159,5.0,90000.0
1,28.0,0,2,17,3.0,65000.0
2,45.0,1,3,130,15.0,150000.0
3,36.0,0,1,101,7.0,60000.0
4,52.0,1,2,22,20.0,200000.0


In [ ]:
le = LabelEncoder()
df.Gender=le.fit_transform(df.Gender)
df['Job Title']=le.fit_transform(df['Job Title'])

In [ ]:

df.dropna(subset=['Education Level'], inplace=True)
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Education Level'], how="all")

In [ ]:
education_mapping = {
    "Bachelor's": 1,
    "Master's": 2,
    "PhD": 3
}
df['Education Level'] = df['Education Level'].map(education_mapping).astype('int32')

In [ ]:
X = df.drop(columns=['Salary'])
y = df['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Rozmiar zbioru treningowego:", len(X_train))
print("Rozmiar zbioru testowego:", len(X_test))

Rozmiar zbioru treningowego: 261
Rozmiar zbioru testowego: 112


In [ ]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,1,159,5.0,90000.0
1,28.0,0,2,17,3.0,65000.0
2,45.0,1,3,130,15.0,150000.0
3,36.0,0,1,101,7.0,60000.0
4,52.0,1,2,22,20.0,200000.0


In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

y_predicted = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
print("Błąd średni absolutny:", mae)

Błąd średni absolutny: 11377.183126703107


In [ ]:
from sklearn.metrics import median_absolute_error
median_absolute_error(y_test, y_predicted)

8320.821301534219

In [ ]:
model_Huber = HuberRegressor()

model_Huber.fit(X_train, y_train)

y_H_predicted = model_Huber.predict(X_test)

mae_H = mean_absolute_error(y_test, y_H_predicted)
med_ae_H = median_absolute_error(y_test, y_H_predicted)
print("Błąd średni absolutny:", mae_H)
print("Błąd medianowy absolutny:", med_ae_H)

Błąd średni absolutny: 10949.56393027116
Błąd medianowy absolutny: 7460.859800526014


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# calculating VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)

               feature        VIF
0                  Age  18.985818
1               Gender   2.056672
2      Education Level   8.341344
3            Job Title   4.864828
4  Years of Experience   7.764759
